# Stacking Meta-Learner Experiment

**CRITICAL INSIGHT**: ALL model families tested follow the SAME CV-LB relationship:
- LB = 4.23×CV + 0.0533 (R²=0.981)
- Intercept (0.0533) > Target (0.0347) → Current approach CANNOT reach target
- MLP, LGBM, Ridge, GP, k-NN - ALL ON THE SAME LINE

**Hypothesis**: Stacking might have a different CV-LB relationship because:
1. The meta-learner learns to combine predictions optimally
2. It can weight models differently for different inputs
3. Research shows stacking can reduce generalization gap

**Implementation**:
- Base models: MLP, LGBM, GP, Ridge
- Generate out-of-fold predictions for each base model
- Train Ridge meta-learner on OOF predictions
- Use nested CV to avoid leakage

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

Simple feature dimension: 18


In [6]:
# Simple MLP for base model
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=3, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, output_dim)
        )
        
    def forward(self, x):
        return self.net(x)

print('SimpleMLP defined')

SimpleMLP defined


In [7]:
# Stacking Model with Meta-Learner
class StackingModel:
    """Stacking ensemble with Ridge meta-learner.
    
    Base models:
    - MLP (neural network)
    - LGBM (gradient boosting)
    - GP (Gaussian Process)
    - Ridge (linear regression)
    
    Meta-learner: Ridge regression on base model predictions
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.full_featurizer = FullFeaturizer(mixed=(data == 'full'))
        self.simple_featurizer = SimpleFeaturizer(mixed=(data == 'full'))
        
        # Scalers
        self.full_scaler = StandardScaler()
        self.simple_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        
        # Base models
        self.mlp = None
        self.lgbm_models = []
        self.gp_models = []
        self.ridge_models = []
        
        # Meta-learner (one per target)
        self.meta_learners = []
        
    def train_model(self, X, Y):
        """Train stacking model with nested CV for OOF predictions."""
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        # Scale
        X_full_scaled = self.full_scaler.fit_transform(X_full)
        X_simple_scaled = self.simple_scaler.fit_transform(X_simple)
        Y_scaled = self.y_scaler.fit_transform(Y_np)
        
        n_samples = len(X)
        
        # Generate OOF predictions for each base model using 5-fold CV
        from sklearn.model_selection import KFold
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        
        # Initialize OOF prediction arrays
        oof_mlp = np.zeros((n_samples, 3))
        oof_lgbm = np.zeros((n_samples, 3))
        oof_gp = np.zeros((n_samples, 3))
        oof_ridge = np.zeros((n_samples, 3))
        
        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_full_scaled)):
            X_train_full = X_full_scaled[train_idx]
            X_val_full = X_full_scaled[val_idx]
            X_train_simple = X_simple_scaled[train_idx]
            X_val_simple = X_simple_scaled[val_idx]
            Y_train = Y_scaled[train_idx]
            
            # Train MLP
            mlp = SimpleMLP(self.full_featurizer.feats_dim, hidden_dim=64, output_dim=3, dropout=0.2).to(device)
            self._train_mlp_fold(mlp, X_train_full, Y_train)
            mlp.eval()
            with torch.no_grad():
                X_val_tensor = torch.tensor(X_val_full, dtype=torch.double).to(device)
                oof_mlp[val_idx] = mlp(X_val_tensor).cpu().numpy()
            
            # Train LGBM (one per target)
            for i in range(3):
                lgbm_model = lgb.LGBMRegressor(
                    n_estimators=100, learning_rate=0.05, max_depth=4,
                    num_leaves=15, min_child_samples=10, reg_alpha=0.1, reg_lambda=0.1,
                    random_state=42, verbose=-1
                )
                lgbm_model.fit(X_train_full, Y_train[:, i])
                oof_lgbm[val_idx, i] = lgbm_model.predict(X_val_full)
            
            # Train GP (one per target) - use simple features
            for i in range(3):
                kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
                gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, normalize_y=True, random_state=42)
                gp.fit(X_train_simple, Y_train[:, i])
                oof_gp[val_idx, i] = gp.predict(X_val_simple)
            
            # Train Ridge (one per target)
            for i in range(3):
                ridge = Ridge(alpha=1.0)
                ridge.fit(X_train_full, Y_train[:, i])
                oof_ridge[val_idx, i] = ridge.predict(X_val_full)
        
        # Stack OOF predictions: shape (n_samples, 12) for 4 models x 3 targets
        stacked_features = np.hstack([oof_mlp, oof_lgbm, oof_gp, oof_ridge])
        
        # Train meta-learner (Ridge) on stacked features
        self.meta_learners = []
        for i in range(3):
            meta = Ridge(alpha=1.0)
            meta.fit(stacked_features, Y_scaled[:, i])
            self.meta_learners.append(meta)
        
        # Train final base models on full data for prediction
        self._train_final_models(X_full_scaled, X_simple_scaled, Y_scaled)
    
    def _train_mlp_fold(self, mlp, X_scaled, Y_scaled):
        X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_scaled, dtype=torch.double).to(device)
        
        dataset = TensorDataset(X_tensor, Y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        optimizer = torch.optim.AdamW(mlp.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
        
        target_weights = torch.tensor([1.0, 1.0, 2.0], dtype=torch.double).to(device)
        
        mlp.train()
        for epoch in range(100):
            for X_batch, Y_batch in loader:
                optimizer.zero_grad()
                pred = mlp(X_batch)
                loss = ((pred - Y_batch) ** 2 * target_weights).mean()
                loss.backward()
                optimizer.step()
            scheduler.step()
    
    def _train_final_models(self, X_full_scaled, X_simple_scaled, Y_scaled):
        """Train final base models on full training data."""
        # MLP
        self.mlp = SimpleMLP(self.full_featurizer.feats_dim, hidden_dim=64, output_dim=3, dropout=0.2).to(device)
        self._train_mlp_fold(self.mlp, X_full_scaled, Y_scaled)
        
        # LGBM
        self.lgbm_models = []
        for i in range(3):
            lgbm_model = lgb.LGBMRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=4,
                num_leaves=15, min_child_samples=10, reg_alpha=0.1, reg_lambda=0.1,
                random_state=42, verbose=-1
            )
            lgbm_model.fit(X_full_scaled, Y_scaled[:, i])
            self.lgbm_models.append(lgbm_model)
        
        # GP
        self.gp_models = []
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=2, normalize_y=True, random_state=42)
            gp.fit(X_simple_scaled, Y_scaled[:, i])
            self.gp_models.append(gp)
        
        # Ridge
        self.ridge_models = []
        for i in range(3):
            ridge = Ridge(alpha=1.0)
            ridge.fit(X_full_scaled, Y_scaled[:, i])
            self.ridge_models.append(ridge)
    
    def predict(self, X):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Scale
        X_full_scaled = self.full_scaler.transform(X_full)
        X_simple_scaled = self.simple_scaler.transform(X_simple)
        
        # Get base model predictions
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_pred = self.mlp(X_tensor).cpu().numpy()
        
        lgbm_pred = np.column_stack([lgbm.predict(X_full_scaled) for lgbm in self.lgbm_models])
        gp_pred = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gp_models])
        ridge_pred = np.column_stack([ridge.predict(X_full_scaled) for ridge in self.ridge_models])
        
        # Stack predictions
        stacked_features = np.hstack([mlp_pred, lgbm_pred, gp_pred, ridge_pred])
        
        # Meta-learner prediction
        meta_pred_scaled = np.column_stack([meta.predict(stacked_features) for meta in self.meta_learners])
        
        # Inverse transform
        meta_pred = self.y_scaler.inverse_transform(meta_pred_scaled)
        
        # Clip to valid range
        meta_pred = np.clip(meta_pred, 0, 1)
        
        return torch.tensor(meta_pred)
    
    def predict_with_tta(self, X):
        """Predict with test-time augmentation for mixed solvents."""
        if self.data_type == 'single':
            return self.predict(X)
        
        # For mixed solvents, average predictions from both orderings
        pred1 = self.predict(X)
        
        # Create flipped version
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        pred2 = self.predict(X_flip)
        
        # Average
        return (pred1 + pred2) / 2

print('StackingModel defined with MLP + LGBM + GP + Ridge base models and Ridge meta-learner')

StackingModel defined with MLP + LGBM + GP + Ridge base models and Ridge meta-learner


In [8]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

Single solvent data: 656 samples, 24 unique solvents
Full data: 1227 samples, 13 unique ramps


In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = StackingModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [01:06, 66.04s/it]

2it [02:09, 64.65s/it]

3it [03:08, 61.76s/it]

4it [04:06, 60.52s/it]

...

21it [23:25, 68.98s/it]

22it [24:32, 68.53s/it]

23it [25:37, 67.38s/it]

24it [26:46, 68.05s/it]

24it [26:46, 66.96s/it]

In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = StackingModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [04:11, 251.10s/it]

2it [08:26, 253.57s/it]

3it [12:47, 257.23s/it]

4it [16:52, 252.10s/it]

...

10it [42:30, 266.88s/it]

11it [46:45, 263.36s/it]

12it [51:10, 263.83s/it]

13it [55:49, 268.28s/it]

13it [55:49, 257.62s/it]

In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [12]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== STACKING MODEL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_032 (best CV, GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194')
print(f'exp_045 (STACKING): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_032!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_032')
    
# Estimate LB using old relationship
estimated_lb = 4.23 * overall_mse + 0.0533
print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
print(f'Best LB so far: 0.0877')
print(f'Target: 0.0347')
print(f'\nKey question: Does Stacking have a DIFFERENT CV-LB relationship?')
print(f'If Stacking has lower intercept, it might reach the target even with worse CV.')


=== STACKING MODEL CV SCORE ===
Single Solvent MSE: 0.010255 (n=656)
Full Data MSE: 0.009865 (n=1227)
Overall MSE: 0.010001

=== COMPARISON ===
exp_032 (best CV, GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194
exp_045 (STACKING): CV 0.010001

✗ WORSE: 22.05% worse than exp_032

Estimated LB (using old relationship): 0.0956
Best LB so far: 0.0877
Target: 0.0347

Key question: Does Stacking have a DIFFERENT CV-LB relationship?
If Stacking has lower intercept, it might reach the target even with worse CV.
